# Práctica Calificada 1
Alumna: Janice Katherine Escobedo Vasquez
## Problema 1
El problema del puzzle 8 que permita ingresar diferentes heurísticas  (2 puntos)

In [1]:
import random
import numpy as np

Definamos los posibles movimientos por indices. Si consideramos las posiciones del juego como siguen

| 0 | 1| 2 |
| ----- | --- | ----- |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

Entonces
- desde la posicion 0 solo nos podemos DERECHA, ABAJO;es decir, a los indices 1 y 3
- desde la posicion 7 nos podemos mover ARRIBA, IZQUIERDA,  DERECHA; es decir, a los indices 4,6 y 8
- desde la posicion 4 nos podemos mover a los cuatro lados con indices 1,3,5,7
De acuerdo a eso definimos el diccionario poss_moves

In [2]:
# Posibles movimientos que se pueden dar por posicion; es decir, cuando me encuentre en la posici
poss_moves = {
        0: [1, 3],
        1: [0, 2, 4],
        2: [1, 5],
        3: [0, 4, 6],
        4: [1, 3, 5, 7],
        5: [2, 4, 8],
        6: [3, 7],
        7: [4, 6, 8],
        8: [5, 7]
}

Definiendo Heuristicas

In [3]:
def Heuristica1(state): # Heuristica 1 es la cantidad de fichas descolocadas
    valid = state==pos_final
    h = (valid==False).sum()
    if list(state).index(0) == 4:
        return h
    else: # aquí el 4 se encontraría descolocado, pero para la heuristica el 0
          # no cuenta como casilla descolocada así no se encuentre en la posicion del medio
        return h-1

def Heuristica2(state): # Heuristica 2 es la distancia Manhattan
    h =0 # la distancia se inicializa con 0
    valid = state==pos_final
    (inx,iny) = np.where(valid.reshape((3,3))==False) # almacenamos indices de casillas descolocadas

    for k in range(len(inx)): # tener en cuenta que len(inx) = len(iny)
        (indx_c, indy_c) = np.where(pos_final.reshape((3,3))== state.reshape((3,3))[inx[k],iny[k]])
        if state.reshape((3,3))[inx[k],iny[k]] != 0: # el 0 no cuenta como ficha incorrecta
            h = h + abs(indx_c[0]- inx[k]) + abs(indy_c[0]- iny[k]) # almacena las distancias
    return h

In [4]:
# Realiza el cambio de estado intercambiando la ficha de entrada por el casillero vacío
def Possible_movement(curr_state,pos_empty,index, Heuristic):
    state_tmp = curr_state.copy()
    state_tmp[pos_empty] = state_tmp[index]
    state_tmp[index] = 0
    return state_tmp, Heuristic(state_tmp)

# Devuelve los nodos hijos de un posible estado, con su respectiva heuristica
def Puzzle(state, Heuristic):
    # buscamos la posicion del casilero vacio (0)
    pos_empty = list(state).index(0)

    # h_1 es la heuristica de los ndos hijos posibles
    h_1 = np.array([], dtype = np.dtype(np.int32))

    # poss_child almacena todos los estados posibles que se pueden dar del estado pacre 'state'
    poss_child = []

    for i in poss_moves[pos_empty]: # el valor de la llave devuelve los cambios posibles que se dan por posicion
        poss_child.append(Possible_movement(state,pos_empty,i, Heuristic)[0])
        h_1 = np.append(h_1,Possible_movement(state,pos_empty,i, Heuristic)[1])
    return poss_child, h_1

# Cuando aplicando la heuristica a los nodos hijos y escogiendo el mínimo este se
# da en más de un nodo, entonces juntamos los nodos hijos de todos los estados donde se dé el mínimo
def Child(poss_acum,h_acum, Heuristic):
    poss_child = poss_acum.copy()
    h_1 = h_acum.copy()
    h_curr = min(h_1)
    # Indices de los estados donde se da la minima heuristica
    poss, = np.where((h_1<=h_curr)) and np.where(h_1 == h_curr)
    poss_acum = []
    h_acum = []

    # acumulando todos los estados y heuristicas
    for i in poss:
        poss_acum = poss_acum + Puzzle(poss_child[i], Heuristic)[0]
        h_acum.extend(Puzzle(poss_child[i], Heuristic)[1])
            # h_acum = h_acum + Puzzle(new_poss[i], Heuristica1)[1]
    return poss_acum,h_acum

# Imprimir el estado actual con su rspectiva heuristica
def Print_sol(state,h,i):
    print("Movement ",i, "\n",state.reshape((3,3)))
    print("Heuristica: ", h, "\n")

Para este ejemplo nuestro input es un array que almacena las posiciones del puzzle. \
Ejm: Si nuestro input de estado inicial es
$$[2,8,3,1,6,4,7,0,5]$$
y el input del estado final es
$$ [1,2,3,8,0,4,7,6,5] $$
Entonces el juego comienza de la siguiente manera

| 2 | 8| 3 |
| ----- | --- | ----- |
| 1 | 6 | 4 |
| 7 |  | 5 |

y la posicion final es

| 1 | 2 | 3 |
| ----- | --- | ----- |
| 8 |  | 4 |
| 7 | 6 | 5 |

In [5]:
state = np.array([2,8,3,1,6,4,7,0,5])
pos_final = np.array([1,2,3,8,0,4,7,6,5])

def Puzzle_main(state_0,pos_final, Heuristic):
    poss_acum = [state_0] # El camino comienza con el estado inicial
    h_acum = [Heuristica1(state_0)]
    k = 0
    Print_sol(poss_acum[0], h_acum[0],k)

    while 0 not in h_acum: # mientras no lleguemos a la posicion final
        k = k+1
        poss_acum, h_acum = Child(poss_acum, h_acum, Heuristic)
        poss_1, = np.where((h_acum==min(h_acum)))

        Print_sol(poss_acum[poss_1[0]], h_acum[poss_1[0]],k)

In [6]:
Puzzle_main(state,pos_final, Heuristica1)

Movement  0 
 [[2 8 3]
 [1 6 4]
 [7 0 5]]
Heuristica:  4 

Movement  1 
 [[2 8 3]
 [1 0 4]
 [7 6 5]]
Heuristica:  3 

Movement  2 
 [[2 0 3]
 [1 8 4]
 [7 6 5]]
Heuristica:  3 

Movement  3 
 [[0 2 3]
 [1 8 4]
 [7 6 5]]
Heuristica:  2 

Movement  4 
 [[1 2 3]
 [0 8 4]
 [7 6 5]]
Heuristica:  1 

Movement  5 
 [[1 2 3]
 [8 0 4]
 [7 6 5]]
Heuristica:  0 



In [7]:
Puzzle_main(state,pos_final, Heuristica2)

Movement  0 
 [[2 8 3]
 [1 6 4]
 [7 0 5]]
Heuristica:  4 

Movement  1 
 [[2 8 3]
 [1 0 4]
 [7 6 5]]
Heuristica:  4 

Movement  2 
 [[2 0 3]
 [1 8 4]
 [7 6 5]]
Heuristica:  3 

Movement  3 
 [[0 2 3]
 [1 8 4]
 [7 6 5]]
Heuristica:  2 

Movement  4 
 [[1 2 3]
 [0 8 4]
 [7 6 5]]
Heuristica:  1 

Movement  5 
 [[1 2 3]
 [8 0 4]
 [7 6 5]]
Heuristica:  0 



In [ ]:
state = np.array(list(map(int, input("Enter the initial state of the puzzle (use 0 for the blank space): ").split())))
pos_final =np.array(list(map(int, input("Enter the goal state of the puzzle: ").split())))
Puzzle_main(state,pos_final, Heuristica1)

Enter the initial state of the puzzle (use 0 for the blank space): 2 8 3 1 6 4 7 0 5
Enter the goal state of the puzzle: 1 2 3 8 0 4 7 6 5
Movement  0 
 [[2 8 3]
 [1 6 4]
 [7 0 5]]
Heuristica:  4 

Movement  1 
 [[2 8 3]
 [1 0 4]
 [7 6 5]]
Heuristica:  3 

Movement  2 
 [[2 0 3]
 [1 8 4]
 [7 6 5]]
Heuristica:  3 

Movement  3 
 [[0 2 3]
 [1 8 4]
 [7 6 5]]
Heuristica:  2 

Movement  4 
 [[1 2 3]
 [0 8 4]
 [7 6 5]]
Heuristica:  1 

Movement  5 
 [[1 2 3]
 [8 0 4]
 [7 6 5]]
Heuristica:  0 



## Problema 2
El problema de las 4 reinas\
a) Usando el método de búsqueda en profundidad DFS (1.5 puntos)

In [8]:
import numpy as np

In [9]:
# Heuristica: conteo de reinas
def Heuristica2(state):
    (n_queenx,__) = np.where(state=='Q')
    return len(n_queenx)

In [10]:
# Asigna con 'X' los movimientos donde la Reina representada por 'Q' no se debe mover

def Prohibited_movements(state):
    state_curr = state.copy()
    (qx,qy) =np.where(state_curr=='Q') # Ubicacion de reinas
    for k in range(len(qx)): # por cada reina que encuentre en el estado
        for i in range(4):
            for j in range(4):
                if state_curr[i,j] == '': # Solo pinta los espacios vacios
                    if i != qx[k] and j !=qy[k]:
                        # (i+j == qx[k]+qy[k]) es la condicion que cumplen los indices de los elementos de la diagonal reversa
                        if i+j == qx[k]+qy[k]: # Para la Diagonal reversa
                            state_curr[i,j] ="X"
                        if i-j == qx[k]-qy[k]: # Para la Diagonal
                            state_curr[i,j] ="X"
                    if j != qy[k]:
                        state_curr[qx[k],j] = "X" # Para la fila
                    if i != qx[k]:
                        state_curr[i,qy[k]] = "X" # Para la columna
    return state_curr

In [11]:
# Esta función acumula los estados recorridos en la lista Closed, aqui consideramos 2 cosas
# - Simetria: Pasar al estado 'state' es equivalente a pasar a 'state' transpuesto
# - Rotación: Pasar al estado 'state' es equivalente a pasar a 'state' rotado 90, 180 , 270

def Add_to_closed(Closed,state_1):
    Closed.append(state_1)
    transpose = state_1.T
    Closed.append(transpose)
    rotate = np.rot90(state_1)
    for k in range(3):
        Closed.append(rotate)
        rotate = np.rot90(rotate)

    return list(np.unique(Closed, axis=0)) # Al final si algun estado se repite lo consideramos solo una vez

In [13]:
# Funcion que devuelve los nodos hijo del estado 'initial'

def Get_children(Closed,initial):
    initial_p = Prohibited_movements(initial)
    initial_p
    # Consideramos los indices de los espacios vacios (posibles opciones de poner la siguiente reina)
    (inx,iny) =np.where(initial_p=='')

    # children almacena los nodos hijos del estado 'initial'
    children = []
    # h almacena las heuristicas de los nodos hijo del estado 'initial'
    h = []

    for i in range(len(inx)):
        initial2 = initial.copy()
        # Asignamos a la siguiente reina el los todos los posibles espacios en blanco
        initial2[inx[i],iny[i]] = "Q"

        # valid es una lista de arrays en 2d con tipo de dato booleano
        valid = initial2 == Closed

        # Si la suma de elementos de los elementos de valid es 16 significa que
        # el estado initial2 esta dentro de los estados cerrados, lo que deseamos
        # es que se considere un nuevo hijo si es que el estado initial2 no está
        # dentro de Closed; es decir, la suma no es 16
        if 16 not in [valid[k].sum() for k in range(len(valid))]: # si no esta en Closed
            children.append(initial2)
            h.append(Heuristica2(initial2))
        # Luego de tener los nodos hijo actualizamos nuestro conjunto cerrado
        Closed = Add_to_closed(Closed,initial2)
    return Closed, children, h

In [14]:
def PrintHijos(children):
    for i in range(len(children)):
        print(children[i])

In [21]:
def Cuatro_Reinas(Closed,initial, sol = None): # Arbol_busqueda, costo, camino
    # Para el estado initial,
    Closed, children, h = Get_children(Closed,initial)

    # Si tiene hijos el estado initial, entonces ver a los hijos
    if len(children)>0:
        print('\nPara el estado: \n',initial,'\nLos nodos hijos posibles son:')
        PrintHijos(children)

        # Si alguno de ellos tiene las 4 reinas entonces habremos encontrado solucion,
        # de igual manera se inprimirá el árbol de búsqueda
        for pos in range(len(children)):
            if h[pos]==4:
                sol = children[pos]
                print('¡ACABAMOS DE ENCONTRAR UNA SOLUCION!\n')
                return sol
            else:
                sol = Cuatro_Reinas(Closed,children[pos],sol)
    return sol

In [22]:
# Comenzamos con un estado inicial de un tablero vacio
initial = np.zeros((4,4), dtype = np.dtype("<U3"))
# Y con el Cerrado igual al tablero vacio
Closed = [initial]
# Vemos si hay solucion
sol = Cuatro_Reinas(Closed,initial)


Para el estado: 
 [['' '' '' '']
 ['' '' '' '']
 ['' '' '' '']
 ['' '' '' '']] 
Los nodos hijos posibles son:
[['Q' '' '' '']
 ['' '' '' '']
 ['' '' '' '']
 ['' '' '' '']]
[['' 'Q' '' '']
 ['' '' '' '']
 ['' '' '' '']
 ['' '' '' '']]
[['' '' 'Q' '']
 ['' '' '' '']
 ['' '' '' '']
 ['' '' '' '']]
[['' '' '' '']
 ['' 'Q' '' '']
 ['' '' '' '']
 ['' '' '' '']]

Para el estado: 
 [['Q' '' '' '']
 ['' '' '' '']
 ['' '' '' '']
 ['' '' '' '']] 
Los nodos hijos posibles son:
[['Q' '' '' '']
 ['' '' 'Q' '']
 ['' '' '' '']
 ['' '' '' '']]
[['Q' '' '' '']
 ['' '' '' 'Q']
 ['' '' '' '']
 ['' '' '' '']]
[['Q' '' '' '']
 ['' '' '' '']
 ['' '' '' 'Q']
 ['' '' '' '']]

Para el estado: 
 [['Q' '' '' '']
 ['' '' 'Q' '']
 ['' '' '' '']
 ['' '' '' '']] 
Los nodos hijos posibles son:
[['Q' '' '' '']
 ['' '' 'Q' '']
 ['' '' '' '']
 ['' 'Q' '' '']]

Para el estado: 
 [['Q' '' '' '']
 ['' '' '' 'Q']
 ['' '' '' '']
 ['' '' '' '']] 
Los nodos hijos posibles son:
[['Q' '' '' '']
 ['' '' '' 'Q']
 ['' '' '' '']
 ['

In [17]:
print('La solución de las 4 reinas es\n',sol)

La solución de las 4 reinas es
 [['' '' 'Q' '']
 ['Q' '' '' '']
 ['' '' '' 'Q']
 ['' 'Q' '' '']]


$b)$ Usando el método de búsqueda de costo uniforme UCS (1.5 puntos)

In [18]:
# Usamos la heuristica de la suma de reinas + los espacios disponibles vacíos
# Cuando encontremos el óptimo h = 4 + 0 = 4
def Heuristica1(state):
    (emptyx,__) = np.where(state=='') # np.where(state=='Q') or
    (n_queenx,__) = np.where(state=='Q')
    return len(emptyx) + len(n_queenx)

## Problema 3
El problema de Romania\
$a)$ Usando el método de búsqueda en profundidad DFS (1.5 puntos)

In [19]:
# Algoritmo DFS

def DFS(grafo, state, final_state, costo_final, Closed=set(), camino=[]):
    camino = camino + [state]
    if state == final_state:
        return camino, costo_final
    # Agregamos el estado al conjunto Closed
    Closed.add(state)

    # Para todos los posibles conexiones que tiene el estaso
    for child, cost in grafo[state]:
        # Si la cuidad no se ha recorrido
        if child not in Closed:
            # Incrementamos el costo
            costo_final = costo_final + cost
            # Se define nuevo camino
            new_path, costo_final = DFS(grafo, child, final_state, costo_final, Closed, camino)
            return new_path, costo_final

    return [], costo_final

# Definimos el Grafo donde cada noto es una ciudad_origen y cada valor es una lista de (cuidad_destino, costo)
Rumania= {
    'A': [('Z', 75), ('S', 140), ('T', 118)],
    'Z': [('A', 75), ('O', 71)],
    'O': [('Z', 71), ('S', 151)],
    'T': [('A', 118),('L', 111)],
    'L': [('T', 111), ('M', 70)],
    'S': [('O', 151), ('A', 140), ('F', 99), ('R', 80)],
    'M': [('L', 70), ('D', 75)],
    'D': [('M', 75), ('C', 120)],
    'C': [('D', 120), ('R', 146), ('P', 138)],
    'R': [('S', 80), ('P', 97), ('C', 146)],
    'P': [('R', 97), ('C', 138), ('B', 101)],
    'F': [('S', 99), ('B', 211)],
    'B': [('P', 101), ('G', 90), ('F', 211), ('U', 85)],
    'G': [('B', 90)],
    'U': [('B', 85), ('V', 142), ('H', 98)],
    'V': [('U', 142), ('I', 92)],
    'I': [('V', 92), ('N', 87)],
    'N': [('I', 87)],
    'H': [('U', 98), ('E', 86)],
    'E': [('H', 86)]
}

ciudad_inicial = 'A'
cuidad_final = 'B'
camino, costo_final = DFS(Rumania, ciudad_inicial, cuidad_final, 0)

if camino:
    print("Camino encontrado:\n", camino)
    print("El costo total:\n", costo_final)
else:
    print("No se encontró un camino desde", ciudad_inicial, "a", cuidad_final)

Camino encontrado:
 ['A', 'Z', 'O', 'S', 'F', 'B']
El costo total:
 607


$b)$ Usando el método de búsqueda de costo uniforme UCS (1.5 puntos)

In [20]:
import heapq # libreria para el uso de colas

def UCS(grafo, start, goal):
    frontera = [(0, start, [])]  # almacena (costo acumulado, nodo, camino)
    Closed = set()
    k = 1
    while frontera:
        print('Paso ',k)
        print(frontera)
        k = k + 1
        cost, current_node, path = heapq.heappop(frontera) # saca el de menor costo
        if current_node == goal:
            return cost, path + [current_node], Closed, frontera
        # Recorre caminos que no estén dentro de Closed
        if current_node not in Closed:
            Closed.add(current_node)
            # Recorre los valores de la ciudad actual
            for neighbor, neighbor_cost in grafo[current_node]:
                if neighbor not in Closed:
                    # Añade a la cola frontera el posible vecino
                    heapq.heappush(frontera, (cost + neighbor_cost, neighbor, path + [current_node]))
    return None, None

Rumania= {
    'A': [('T', 118), ('Z', 75), ('S', 140)],
    'Z': [('A', 75), ('O', 71)],
    'O': [('Z', 71), ('S', 151)],
    'T': [('A', 118),('L', 111)],
    'L': [('T', 111), ('M', 70)],
    'M': [('L', 70), ('D', 75)],
    'D': [('M', 75), ('C', 120)],
    'C': [('D', 120), ('R', 146), ('P', 138)],
    'R': [('S', 80), ('P', 97), ('C', 146)],
    'P': [('R', 97), ('C', 138), ('B', 101)],
    'F': [('S', 99), ('B', 211)],
    'S': [('O', 151), ('A', 140), ('F', 99), ('R', 80)],
    'B': [('P', 101), ('G', 90), ('F', 211), ('U', 85)],
    'G': [('B', 90)],
    'U': [('B', 85), ('V', 142), ('H', 98)],
    'V': [('U', 142), ('I', 92)],
    'I': [('V', 92), ('N', 87)],
    'N': [('I', 87)],
    'H': [('U', 98), ('E', 86)],
    'E': [('H', 86)]
}

ciudad_inicial = 'A'
cuidad_final = 'B'
cost, path, Closed, frontera = UCS(Rumania, ciudad_inicial, cuidad_final)

if path:
    print("\nCamino recorrido:\n", path)
    print("\nMenor costo total del camino:\n", cost)
    print('\nNodos recorridos: \n', Closed)
else:
    print("No se encontró un camino desde", ciudad_inicial, "a", cuidad_final)

Paso  1
[(0, 'A', [])]
Paso  2
[(75, 'Z', ['A']), (118, 'T', ['A']), (140, 'S', ['A'])]
Paso  3
[(118, 'T', ['A']), (140, 'S', ['A']), (146, 'O', ['A', 'Z'])]
Paso  4
[(140, 'S', ['A']), (146, 'O', ['A', 'Z']), (229, 'L', ['A', 'T'])]
Paso  5
[(146, 'O', ['A', 'Z']), (220, 'R', ['A', 'S']), (291, 'O', ['A', 'S']), (239, 'F', ['A', 'S']), (229, 'L', ['A', 'T'])]
Paso  6
[(220, 'R', ['A', 'S']), (229, 'L', ['A', 'T']), (291, 'O', ['A', 'S']), (239, 'F', ['A', 'S'])]
Paso  7
[(229, 'L', ['A', 'T']), (239, 'F', ['A', 'S']), (291, 'O', ['A', 'S']), (317, 'P', ['A', 'S', 'R']), (366, 'C', ['A', 'S', 'R'])]
Paso  8
[(239, 'F', ['A', 'S']), (299, 'M', ['A', 'T', 'L']), (291, 'O', ['A', 'S']), (366, 'C', ['A', 'S', 'R']), (317, 'P', ['A', 'S', 'R'])]
Paso  9
[(291, 'O', ['A', 'S']), (299, 'M', ['A', 'T', 'L']), (317, 'P', ['A', 'S', 'R']), (366, 'C', ['A', 'S', 'R']), (450, 'B', ['A', 'S', 'F'])]
Paso  10
[(299, 'M', ['A', 'T', 'L']), (366, 'C', ['A', 'S', 'R']), (317, 'P', ['A', 'S', 'R']), (4